In [1]:
import numpy as np

from fixed_arith import fixed_mul, relu, fixed2float, unsigned_fixed_add, fixed_add, \
     dec_to_bin, dec_to_bin_array, int_digit, fixed_add_array
from fixed_arith import fixed_mul
from fixed_conv import conv_forward#, fixed_conv

# def fixed_add_array(adder_array, integer=8):
#     """ given a list of fixed point 16, return the sum
#     x_arr: list of fp16 strings
#     integer: number of digits
#     return:
#         a single fp16
#     """
#     if len(adder_array) == 0:
#         return '0' * 16
#     elif len(adder_array) == 1:
#         return adder_array[0]
#     else:
#         last_ele = adder_array[0]
        
#         for i in range(1, len(adder_array)):
#             this_carry, temp_result = unsigned_fixed_add(adder_array[i], last_ele)
# #             if this_carry:
# #                 raise Exception("overflow detected in function 'fixed_add_array'")
#             last_ele = temp_result
        
#         return last_ele
    
def fixed_conv(x, w, b, int_x=8, int_w=8, int_b=8):
    """ fixed point conv
    pad width = 1 -> for vgg only
    stride = 1 -> for vgg only
    
    x: 3d array of 16fp strings (height, width, channels)
    w: 4d array of 16fp strings (num_of_filters, filter_height, filter_width, channels),
        where filter_height = filter_width = 3
    b: 1d array (num_of_filters, )
    
    int_x/w/b: how many digits for integer parts
    
    result:
        out: 3d array, has the same shape as x
    """
    height, width, channels = x.shape
    num_filters, filter_height, filter_width, _ = w.shape
    
    # create result list
    result = a = np.zeros((height, width, num_filters), dtype='<U16')
    
    # padding x
    x_pad = np.zeros((height + 2, width + 2, channels), dtype='<U16')
    x_pad[1:-1,1:-1,:] = x
    x_pad[0,:] = '0' * 16
    x_pad[-1,:] = '0' * 16
    x_pad[:,0] = '0' * 16
    x_pad[:,-1] = '0' * 16
    
    for i in range(num_filters):
        for j in range(height):
            for k in range(width):
                # now this corresponding to result[i][j][k]
                adder_array = []
                for fh in range(filter_height):
                    for fw in range(filter_width):
                        for ch in range(channels):
#                             print(w[i, fh, fw, ch], x_pad[j + fh, k + fw, ch])
                            adder_array.append(fixed_mul(w[i, fh, fw, ch], x_pad[j + fh, k + fw, ch], integer_x1=8, integer_x2=8, integer_result=8))
                ############## not implemented yet ###############
                result[j,k,i] = fixed_add_array(adder_array, integer=8)
    
    return result

In [20]:
height = 10
width = 10
channels = 3

num_filters = 16
filter_height = 3
filter_width = 3

x = np.random.randn(height, width, channels) / 1
w = np.random.randn(num_filters, filter_height, filter_width, channels) / 1
b = np.random.randn(num_filters) / 1

print("upper bound: {}".format(max(np.max(abs(x)), np.max(abs(w)), np.max(abs(b)))))

int_part = 8
dec_part = 15 - int_part
fixed_x = np.reshape(np.array(dec_to_bin_array(x, int_digit=int_part, decimal_digit=dec_part)), (height, width, channels))
fixed_w = np.reshape(np.array(dec_to_bin_array(w, int_digit=int_part, decimal_digit=dec_part)), (num_filters, filter_height, filter_width, channels))
fixed_b = np.reshape(np.array(dec_to_bin_array(b, int_digit=int_part, decimal_digit=dec_part)), (num_filters,))

# print(fixed_w)

result = fixed_conv(fixed_x, fixed_w, fixed_b, int_x=dec_part, int_w=dec_part, int_b=dec_part)

upper bound: 2.990212120932118


In [21]:
print(fixed2float('1111111111111111'), int_part)

-0.0078125 8


In [22]:
result_convert_float = np.zeros((height, width, num_filters))

for i in range(height):
    for j in range(width):
        for k in range(num_filters):
            result_convert_float[i][j][k] = fixed2float(result[i][j][k], int_part)


print(result_convert_float)


[[[ -1.2890625   0.8984375   2.3671875 ...   1.765625    2.7265625
     5.2890625]
  [ -3.625      -5.34375    -0.5390625 ...   0.453125    4.390625
    -3.953125 ]
  [ -3.6875     -4.0078125 -11.859375  ...  -6.3828125   4.1484375
   -12.9765625]
  ...
  [  2.9453125  -1.5078125  -0.1484375 ...  -0.3359375  -0.640625
    -4.078125 ]
  [ -0.6953125   3.578125   -2.5       ...  -0.6953125   1.875
     1.4765625]
  [  0.6875      2.         -1.9453125 ...   3.6640625   0.5078125
     1.078125 ]]

 [[ -3.515625    6.65625     4.765625  ...  -2.8828125   3.2578125
     1.171875 ]
  [ -6.25        4.125      -3.1953125 ...   4.0078125   2.8046875
    -0.6875   ]
  [  6.6875     -5.3046875   0.        ...  -2.828125   10.3515625
     3.2734375]
  ...
  [ -1.7734375  -4.8828125  11.3203125 ...  -6.234375   -0.828125
    -8.3671875]
  [ -3.3828125  -0.3984375   2.1484375 ...  -9.3984375   4.4609375
    -0.9609375]
  [ -4.8359375   3.03125    -5.0546875 ...   2.2890625  -2.921875
    -0.3359375

In [23]:
actual_result = conv_forward(np.reshape(x, (1, height, width, channels)), w, b, pad=1, stride=1)
print(actual_result)

[[[[ 2.64637493e-01  2.47901659e-01  3.37148359e+00 ...  2.51530489e+00
     2.28322725e+00  6.84159495e+00]
   [-2.08340925e+00 -5.98509882e+00  4.77984758e-01 ...  1.18998180e+00
     3.98382408e+00 -2.41682183e+00]
   [-2.09635786e+00 -4.60267120e+00 -1.09033800e+01 ... -5.65460762e+00
     3.71174902e+00 -1.15306947e+01]
   ...
   [ 4.59296299e+00 -2.10352398e+00  8.88204649e-01 ...  4.52617534e-01
    -1.09152029e+00 -2.52365152e+00]
   [ 9.06520805e-01  3.01232339e+00 -1.48065511e+00 ...  5.77029980e-02
     1.46907417e+00  3.03445419e+00]
   [ 2.25753778e+00  1.38604691e+00 -9.64377648e-01 ...  4.40713380e+00
     6.55007141e-02  2.62605438e+00]]

  [[-1.96310855e+00  6.11868788e+00  5.79709525e+00 ... -2.18508531e+00
     2.84039431e+00  2.75403999e+00]
   [-4.69145095e+00  3.58082020e+00 -2.11357257e+00 ...  4.84145945e+00
     2.39407112e+00  9.33778838e-01]
   [ 8.38164678e+00 -5.89178802e+00  1.06681524e+00 ... -2.03600207e+00
     9.98967652e+00  4.85250912e+00]
   ...
   